In [1]:
import os
from PIL import Image
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# 1. Normalización de imágenes
def normalize_images(root_folder):
    skipped_folders = []

    for subdir, _, files in os.walk(root_folder):
        unrecognized_format = False
        for file in files:
            filepath = os.path.join(subdir, file)
            filename, ext = os.path.splitext(filepath)
            output_filepath = filename + ".jpg"

            if ext.lower() in [".png", ".bmp", ".tiff", ".gif", ".jpg", ".jpeg"]:
                with Image.open(filepath) as img:
                    img.convert("RGB").save(output_filepath, "JPEG", quality=90)
                    if filepath != output_filepath:
                        os.remove(filepath)
            else:
                unrecognized_format = True

        if unrecognized_format:
            skipped_folders.append(subdir)

    return skipped_folders

# 2. Modelo de reconocimiento facial
def create_face_model(data_folder):
    X = []  # Imágenes
    y = []  # Etiquetas
    label = 0
    class_names = []  # Nombres de las clases

    subfolders = sorted([f.path for f in os.scandir(data_folder) if f.is_dir()])

    for subdir in subfolders:
        class_names.append(os.path.basename(subdir))  # Añadir el nombre de la subcarpeta a class_names
        for file in os.listdir(subdir):
            filepath = os.path.join(subdir, file)
            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                image = cv2.resize(image, (100, 100))
                X.append(image)
                y.append(label)
        label += 1

    X = np.array(X) / 255.0
    X = X.reshape(-1, 100, 100, 1)
    y = np.array(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(label, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=7, validation_data=(X_test, y_test))

    return model, class_names  # Retornar el modelo y la lista de nombres de clases

# 3. Preprocesamiento de la imagen para la predicción
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (100, 100))
    image = image / 255.0
    image = image.reshape(-1, 100, 100, 1)
    return image

# 4. Función de predicción con umbral de probabilidad
def predict_class(image_path, model, threshold=0.8):
    image = preprocess_image(image_path)
    predictions = model.predict(image)[0]
    max_probability = np.max(predictions)
    if max_probability > threshold:
        predicted_class_index = np.argmax(predictions)
        return predicted_class_index, max_probability
    else:
        return None, max_probability

# 5. Función para predecir imágenes en una carpeta
def predict_images_in_folder(folder_path, model, class_names):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                image_path = os.path.join(root, file)
                predicted_class_index, max_probability = predict_class(image_path, model)
                if predicted_class_index is not None:
                    print(f'Imagen: {file}, Clase predicha: {class_names[predicted_class_index]}, Probabilidad: {max_probability * 100:.2f}%')
                else:
                    print(f'Imagen: {file}, No se pudo hacer una predicción confiable. Probabilidad más alta: {max_probability * 100:.2f}%')

if __name__ == "__main__":
    root_folder = "fotos"
    skipped = normalize_images(root_folder)
    print("Carpetas con formatos no reconocidos:", skipped)

    model, class_names = create_face_model(root_folder)  # Ahora también obtienes class_names

    


Carpetas con formatos no reconocidos: []
Epoch 1/7
6/6 [==============================] - 1s 152ms/step - loss: 1.9908 - accuracy: 0.2889 - val_loss: 1.7981 - val_accuracy: 0.1154
Epoch 2/7
6/6 [==============================] - 1s 122ms/step - loss: 1.5684 - accuracy: 0.4500 - val_loss: 1.3029 - val_accuracy: 0.5769
Epoch 3/7
6/6 [==============================] - 1s 123ms/step - loss: 1.0168 - accuracy: 0.6389 - val_loss: 0.8249 - val_accuracy: 0.7308
Epoch 4/7
6/6 [==============================] - 1s 122ms/step - loss: 0.5642 - accuracy: 0.8444 - val_loss: 0.6079 - val_accuracy: 0.7821
Epoch 5/7
6/6 [==============================] - 1s 122ms/step - loss: 0.3664 - accuracy: 0.9167 - val_loss: 0.5625 - val_accuracy: 0.8077
Epoch 6/7
6/6 [==============================] - 1s 122ms/step - loss: 0.2229 - accuracy: 0.9333 - val_loss: 0.4423 - val_accuracy: 0.8205
Epoch 7/7
6/6 [==============================] - 1s 123ms/step - loss: 0.1098 - accuracy: 0.9722 - val_loss: 0.3271 - val_acc

In [2]:
# Carpeta que contiene las imágenes a predecir
images_folder = 'fotos/J'
    
# Llamada a la función para predecir las imágenes en la carpeta
predict_images_in_folder(images_folder, model, class_names)

1/1 [==============================] - 0s 78ms/step
Imagen: 1.jpg, No se pudo hacer una predicción confiable. Probabilidad más alta: 70.27%
1/1 [==============================] - 0s 19ms/step
Imagen: 10.jpg, Clase predicha: J, Probabilidad: 96.33%
1/1 [==============================] - 0s 19ms/step
Imagen: 11.jpg, Clase predicha: J, Probabilidad: 90.74%
1/1 [==============================] - 0s 18ms/step
Imagen: 12.jpg, Clase predicha: J, Probabilidad: 93.15%
1/1 [==============================] - 0s 20ms/step
Imagen: 13.jpg, No se pudo hacer una predicción confiable. Probabilidad más alta: 79.78%
1/1 [==============================] - 0s 20ms/step
Imagen: 14.jpg, Clase predicha: J, Probabilidad: 89.47%
1/1 [==============================] - 0s 18ms/step
Imagen: 15.jpg, Clase predicha: J, Probabilidad: 87.28%
1/1 [==============================] - 0s 18ms/step
Imagen: 16.jpg, Clase predicha: J, Probabilidad: 80.09%
1/1 [==============================] - 0s 17ms/step
Imagen: 17.jpg, Cla

In [3]:
foto = '5.jpg'

In [4]:
# Llamada a la función para predecir la clase de una imagen individual
predicted_class_index, max_probability = predict_class(foto, model)
if predicted_class_index is not None:
    print(f'Clase predicha: {class_names[predicted_class_index]}, Probabilidad: {max_probability * 100:.2f}%')
else:
    print(f'No se pudo hacer una predicción confiable. Probabilidad más alta: {max_probability * 100:.2f}%')

1/1 [==============================] - 0s 19ms/step
Clase predicha: H, Probabilidad: 94.17%


In [5]:
foto2 = '10.jpg'

In [6]:
# Llamada a la función para predecir la clase de una imagen individual
predicted_class_index, max_probability = predict_class(foto2, model)
if predicted_class_index is not None:
    print(f'Clase predicha: {class_names[predicted_class_index]}, Probabilidad: {max_probability * 100:.2f}%')
else:
    print(f'No se pudo hacer una predicción confiable. Probabilidad más alta: {max_probability * 100:.2f}%')

1/1 [==============================] - 0s 19ms/step
Clase predicha: D, Probabilidad: 99.98%
